In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 2
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 2: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style1/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style1/2000_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 200, gamma=0.1, last_epoch=-1)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=2, total_epoch=50, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + 0.1*loss_sp + 0.1*loss_cls

        # measure accuracy and record loss
        losses.update(loss.data.tolist(), inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [20]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
#     adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    scheduler.step()
    
    if (epoch+1)%100 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:1.1758 | MainLoss:0.8821 | Alpha:0.0289 | SPLoss:1.6089 | CLSLoss:1.3284 | AUROC:0.0000
Test | 31/16 | Loss:0.6928 | MainLoss:0.6928 | SPLoss:1.4642 | CLSLoss:1.0657 | AUROC:0.5669
Test | 128/16 | Loss:0.6687 | MainLoss:0.6687 | SPLoss:1.4642 | CLSLoss:1.0657 | AUROC:1.0000

Epoch: [2 | 1000] LR: 0.102000
Train | 16/16 | Loss:0.9043 | MainLoss:0.6904 | Alpha:0.0267 | SPLoss:1.2344 | CLSLoss:0.9043 | AUROC:0.0000
Test | 31/16 | Loss:0.6912 | MainLoss:0.6912 | SPLoss:0.9999 | CLSLoss:0.7435 | AUROC:0.5679
Test | 128/16 | Loss:0.6231 | MainLoss:0.6231 | SPLoss:0.9999 | CLSLoss:0.7435 | AUROC:1.0000

Epoch: [3 | 1000] LR: 0.104000
Train | 16/16 | Loss:0.8312 | MainLoss:0.6849 | Alpha:0.0265 | SPLoss:0.8308 | CLSLoss:0.6326 | AUROC:0.0000
Test | 31/16 | Loss:0.6883 | MainLoss:0.6883 | SPLoss:0.6601 | CLSLoss:0.5235 | AUROC:0.5722
Test | 128/16 | Loss:0.4933 | MainLoss:0.4933 | SPLoss:0.6601 | CLSLoss:0.5235 | AUROC:1.0000

Epoch: [4 | 100

Train | 16/16 | Loss:2.0707 | MainLoss:0.2470 | Alpha:0.0257 | SPLoss:18.0584 | CLSLoss:0.1793 | AUROC:0.0000
Test | 31/16 | Loss:0.3145 | MainLoss:0.3145 | SPLoss:15.7135 | CLSLoss:0.1745 | AUROC:0.9729
Test | 128/16 | Loss:0.1847 | MainLoss:0.1847 | SPLoss:15.7135 | CLSLoss:0.1745 | AUROC:0.9908

Epoch: [27 | 1000] LR: 0.152000
Train | 16/16 | Loss:1.6022 | MainLoss:0.2423 | Alpha:0.0269 | SPLoss:13.4198 | CLSLoss:0.1790 | AUROC:0.0000
Test | 31/16 | Loss:0.2239 | MainLoss:0.2239 | SPLoss:11.2518 | CLSLoss:0.1849 | AUROC:0.9745
Test | 128/16 | Loss:0.2480 | MainLoss:0.2480 | SPLoss:11.2518 | CLSLoss:0.1849 | AUROC:0.9950

Epoch: [28 | 1000] LR: 0.154000
Train | 16/16 | Loss:3.5135 | MainLoss:0.2782 | Alpha:0.0264 | SPLoss:32.1737 | CLSLoss:0.1798 | AUROC:0.0000
Test | 31/16 | Loss:0.2971 | MainLoss:0.2971 | SPLoss:58.7613 | CLSLoss:0.1352 | AUROC:0.9578
Test | 128/16 | Loss:0.5158 | MainLoss:0.5158 | SPLoss:58.7614 | CLSLoss:0.1352 | AUROC:0.9849

Epoch: [29 | 1000] LR: 0.156000
Trai

Train | 16/16 | Loss:7.7671 | MainLoss:0.3578 | Alpha:0.0293 | SPLoss:73.9192 | CLSLoss:0.1732 | AUROC:0.0000
Test | 31/16 | Loss:0.2960 | MainLoss:0.2960 | SPLoss:54.0839 | CLSLoss:0.1700 | AUROC:0.9622
Test | 128/16 | Loss:0.4655 | MainLoss:0.4655 | SPLoss:54.0840 | CLSLoss:0.1700 | AUROC:0.9224

Epoch: [52 | 1000] LR: 0.200000
Train | 16/16 | Loss:5.9670 | MainLoss:0.3535 | Alpha:0.0286 | SPLoss:55.9825 | CLSLoss:0.1526 | AUROC:0.0000
Test | 31/16 | Loss:0.2935 | MainLoss:0.2935 | SPLoss:56.3392 | CLSLoss:0.1525 | AUROC:0.9604
Test | 128/16 | Loss:0.5577 | MainLoss:0.5577 | SPLoss:56.3391 | CLSLoss:0.1525 | AUROC:0.8879

Epoch: [53 | 1000] LR: 0.200000
Train | 16/16 | Loss:4.5900 | MainLoss:0.3059 | Alpha:0.0268 | SPLoss:42.6734 | CLSLoss:0.1684 | AUROC:0.0000
Test | 31/16 | Loss:0.2503 | MainLoss:0.2503 | SPLoss:31.2286 | CLSLoss:0.1404 | AUROC:0.9740
Test | 128/16 | Loss:0.6380 | MainLoss:0.6380 | SPLoss:31.2287 | CLSLoss:0.1404 | AUROC:0.8561

Epoch: [54 | 1000] LR: 0.199998
Trai

Train | 16/16 | Loss:2.2021 | MainLoss:0.2675 | Alpha:0.0262 | SPLoss:19.1717 | CLSLoss:0.1749 | AUROC:0.0000
Test | 31/16 | Loss:0.3285 | MainLoss:0.3285 | SPLoss:13.5812 | CLSLoss:0.1544 | AUROC:0.9662
Test | 128/16 | Loss:0.3981 | MainLoss:0.3981 | SPLoss:13.5812 | CLSLoss:0.1544 | AUROC:0.9285

Epoch: [77 | 1000] LR: 0.199692
Train | 16/16 | Loss:1.8043 | MainLoss:0.2898 | Alpha:0.0270 | SPLoss:14.9852 | CLSLoss:0.1599 | AUROC:0.0000
Test | 31/16 | Loss:0.4684 | MainLoss:0.4684 | SPLoss:33.4177 | CLSLoss:0.1354 | AUROC:0.9555
Test | 128/16 | Loss:1.0943 | MainLoss:1.0943 | SPLoss:33.4177 | CLSLoss:0.1354 | AUROC:0.8717

Epoch: [78 | 1000] LR: 0.199667
Train | 16/16 | Loss:2.9469 | MainLoss:0.2843 | Alpha:0.0278 | SPLoss:26.4666 | CLSLoss:0.1596 | AUROC:0.0000
Test | 31/16 | Loss:0.3410 | MainLoss:0.3410 | SPLoss:19.7371 | CLSLoss:0.1604 | AUROC:0.9669
Test | 128/16 | Loss:0.9937 | MainLoss:0.9937 | SPLoss:19.7371 | CLSLoss:0.1604 | AUROC:0.9337

Epoch: [79 | 1000] LR: 0.199640
Trai

Test | 128/16 | Loss:0.6348 | MainLoss:0.6348 | SPLoss:374.9330 | CLSLoss:0.1849 | AUROC:0.8784

Epoch: [101 | 1000] LR: 0.198817
Train | 16/16 | Loss:0.4304 | MainLoss:0.2599 | Alpha:0.4405 | SPLoss:1.5230 | CLSLoss:0.1817 | AUROC:0.0000
Test | 31/16 | Loss:0.2077 | MainLoss:0.2077 | SPLoss:1.5129 | CLSLoss:0.1757 | AUROC:0.9777
Test | 128/16 | Loss:0.7890 | MainLoss:0.7890 | SPLoss:1.5129 | CLSLoss:0.1757 | AUROC:0.8783

Epoch: [102 | 1000] LR: 0.198769
Train | 16/16 | Loss:0.3628 | MainLoss:0.2126 | Alpha:0.4407 | SPLoss:1.3235 | CLSLoss:0.1787 | AUROC:0.0000
Test | 31/16 | Loss:0.1757 | MainLoss:0.1757 | SPLoss:1.1560 | CLSLoss:0.1732 | AUROC:0.9826
Test | 128/16 | Loss:0.9384 | MainLoss:0.9384 | SPLoss:1.1560 | CLSLoss:0.1732 | AUROC:0.8485

Epoch: [103 | 1000] LR: 0.198719
Train | 16/16 | Loss:0.3240 | MainLoss:0.2012 | Alpha:0.4389 | SPLoss:1.0537 | CLSLoss:0.1734 | AUROC:0.0000
Test | 31/16 | Loss:0.1896 | MainLoss:0.1896 | SPLoss:1.0016 | CLSLoss:0.1700 | AUROC:0.9866
Test | 1

...

Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604

Epoch: [901 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0455 | MainLoss:0.0312 | Alpha:0.4933 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604

Epoch: [902 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0537 | MainLoss:0.0394 | Alpha:0.4919 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604

Epoch: [903 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0571 | MainLoss:0.0428 | Alpha:0.4909 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9919
Test | 128

Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604

Epoch: [926 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0598 | MainLoss:0.0455 | Alpha:0.4898 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604

Epoch: [927 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0647 | MainLoss:0.0504 | Alpha:0.4888 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9602

Epoch: [928 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0699 | MainLoss:0.0555 | Alpha:0.4875 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128

Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9603

Epoch: [951 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0521 | MainLoss:0.0377 | Alpha:0.4928 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604

Epoch: [952 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0583 | MainLoss:0.0440 | Alpha:0.4904 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9921
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9603

Epoch: [953 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0539 | MainLoss:0.0396 | Alpha:0.4909 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9922
Test | 128

Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604

Epoch: [976 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0505 | MainLoss:0.0362 | Alpha:0.4926 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9919
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9603

Epoch: [977 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0515 | MainLoss:0.0371 | Alpha:0.4918 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9605

Epoch: [978 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0572 | MainLoss:0.0429 | Alpha:0.4906 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.0000
Test | 31/16 | Loss:0.1434 | MainLoss:0.1434 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9920
Test | 128

Test | 128/16 | Loss:0.7775 | MainLoss:0.7775 | SPLoss:0.0000 | CLSLoss:0.1432 | AUROC:0.9604
